## Importing libraries

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import datetime as dt
from io import StringIO

### Getting account list

In [2]:
#url for getting the account list
url_base = 'https://api.koios.rest/api/v0/'

In [3]:
def koios_api_pull(url, type='get', headers={"Accept": "text/csv"}, params=None, print_text=False):
    if type == 'get':
        r = requests.get(url, headers = headers, params=params)
    elif type == 'post':
        r = requests.post(url, headers = headers, data=params)
        
    if print_text:
        print(r.text)
        
    df = pd.read_csv(StringIO(r.text), sep=",")
    return df

In [6]:
offset = 0
address_list = pd.DataFrame(columns=["id"])

while True:
    
    try:
        url = url_base + f'account_list?order=id.asc&offset={offset}'
        df = koios_api_pull(url)
        while list(df.columns) != ["id"]:
            df = koios_api_pull(url)
        address_list = pd.concat([address_list, df])
        
        offset += 1000
        
    except:
        break
    
url = url_base + f'account_list?order=id.desc&offset={0}&limit={1000}'
df = koios_api_pull(url)
df = df.sort_values(by=["id"])

df_last_1000 = address_list.tail(1000)
df = df[~np.isin(df["id"], df_last_1000["id"])]

address_list = pd.concat([address_list, df])

In [7]:
address_list 

,id
0,stake17805typy0v6sjluhn4thnuw4h2wdfjmvra0vuj66...
1,stake1780l3f4zdn263ahmyy875n4cy6tnpcdp2n67vpsl...
2,stake1780lxjfcwsuf9qjmqts58huec5lqlssxnz4smvm4...
3,stake1780v0vxp0hg4f9qf2v5hfdllcntphm5yjz4q3jcu...
4,stake17829eha0lrmu40t5hvp79h07xjfvr5uypxyfwrm5...
...,...
4,stake1uyzzzxw74vflyghgkhzw4k0aafatsztstelx6xnc...
3,stake1uyzzzyvc72rxsu7uu2lu3ds3xeu5lc3p9kfgwgxp...
2,stake1uyzzzyx07tlna8feyumseen34294028n0nhpx46g...
1,stake1uyzzzyyzmh8esjvmvdrqre8dddtzty3edslc9cjl...


In [9]:
address_list.to_csv("Stake_Accounts_list.csv", index=False)